<a href="https://colab.research.google.com/github/RajaSuhashKesari/MyDataEngineeringPractices/blob/main/Pyspark%20Programs/30%20days%20challenge%20by%20Seekho%20Big%20data%20institute/3.%20Different_ways_to_remove_duplicates.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## **importing dependencies**

In [12]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
spark = SparkSession.builder.master('local[*]').appName("Different ways to remove duplicates").getOrCreate()

In [13]:
data = [(1, "Alice", 2000),
 (2, "Bob", 3000),
 (3, "Alice", 2000),
 (4, "David", 4000),
 (5, "Alice", 5000),
 (6, "Bob", 3000) ]

In [14]:
columns = ["ID", "Name", "Salary"]

## **Creating DataFrame**

In [15]:
employees_df = spark.createDataFrame(data,columns)
employees_df.show()

+---+-----+------+
| ID| Name|Salary|
+---+-----+------+
|  1|Alice|  2000|
|  2|  Bob|  3000|
|  3|Alice|  2000|
|  4|David|  4000|
|  5|Alice|  5000|
|  6|  Bob|  3000|
+---+-----+------+



## **Method 1 : Using distinct()**

In [16]:
disinct_df = employees_df.distinct()
disinct_df.show()

+---+-----+------+
| ID| Name|Salary|
+---+-----+------+
|  1|Alice|  2000|
|  2|  Bob|  3000|
|  3|Alice|  2000|
|  4|David|  4000|
|  5|Alice|  5000|
|  6|  Bob|  3000|
+---+-----+------+



## **Method 2 : Using dropDuplicates()**

In [17]:
dropduplicates_df = employees_df.dropDuplicates(subset = ['Name','Salary'])
dropduplicates_df.show()

+---+-----+------+
| ID| Name|Salary|
+---+-----+------+
|  2|  Bob|  3000|
|  1|Alice|  2000|
|  5|Alice|  5000|
|  4|David|  4000|
+---+-----+------+



## **Method 3 : Using groupBy() and agg()**

In [19]:
groupby_df = employees_df.groupBy('Name','Salary').agg(first(col('ID')).alias("ID")).select("ID",'Name','Salary')
groupby_df.show()

+---+-----+------+
| ID| Name|Salary|
+---+-----+------+
|  2|  Bob|  3000|
|  1|Alice|  2000|
|  5|Alice|  5000|
|  4|David|  4000|
+---+-----+------+



## **Method 4 : Using Window Function**

In [20]:
from pyspark.sql.window import Window

In [21]:
windowSpec = Window.partitionBy('Name','Salary').orderBy('ID')

In [22]:
df_with_row_num = employees_df.withColumn("Row_Number",row_number().over(windowSpec))
df_with_row_num.show()

+---+-----+------+----------+
| ID| Name|Salary|Row_Number|
+---+-----+------+----------+
|  1|Alice|  2000|         1|
|  3|Alice|  2000|         2|
|  5|Alice|  5000|         1|
|  2|  Bob|  3000|         1|
|  6|  Bob|  3000|         2|
|  4|David|  4000|         1|
+---+-----+------+----------+



In [24]:
filtered_df = df_with_row_num.filter(col('Row_Number') == 1).select("ID","Name","Salary")
filtered_df.show()

+---+-----+------+
| ID| Name|Salary|
+---+-----+------+
|  1|Alice|  2000|
|  5|Alice|  5000|
|  2|  Bob|  3000|
|  4|David|  4000|
+---+-----+------+



In [25]:
spark.stop()